# Delta lakes: creating a sample store

This notebook is intended to create a simple dummy delta lake store compliant with the schema outlined in the ECR validation directory. It leverages `pyspark` and `delta-spark` working within a virtual environment to create an empty table and insert a few dummy rows. It can also be run completely outside of a virtual environment, if desired, in which case only step 4 below need be followed. To configure an environment in order to run this script, perform the following steps:

1. Create a python virtual environment in which to install dependencies and execute the notebook: `python -m venv .venv/`
2. Activate the virtual environment: `source .venv/bin/activate`
3. Install `jupyter` directly in the virtual environment (while it is possible to install kernelspec paths and manipulate environment variables, in practice, simply installing `jupyter` directly without `site-packages` makes importing modules the most seamless): `pip install jupyter`
4. Install the `delta-spark` package in the virtual environment: `pip install delta-spark`. This will automatically install a compatible version of `pyspark` already integrated with the `delta` configuration (it will be version `3.3.2` for most installations).
5. From within the virtual environment, run `jupyter notebook` to launch the navigation page, then select this notebook. The default `Python 3 (ipykernel)` kernel should be displayed in the top right corner of the window, under the `Logout` button.

You can now successfully run this notebook from within your virtual environment!

## Creating the delta lake

First, we'll handle the imports and configuration of the spark builder and delta session (the packages typically assume an interactive running session, but we can use a pre-configured `builder` object to programmatically handle configuration for us).

In [ ]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("dibbs") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

With the packages imported and configured, we can create the schema we'll use to instantiate our delta lake. Schemas in `pyspark` (and therefore `delta-spark`) are defined as `struct` type objects, in which one or more `struct fields` are specified with a name and data type. These can be used to create complex logic around acceptable input formats for the deta, but for now, we'll create some simple string columns designed to hold the data specified in the DIBBS eCR validation schema (found in the message parsing service). After defining the schema, we'll use it to create a blank but formatted spark data frame that we can add to and eventually save as a delta lake.

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType

schema_fields = ["patient_id",
    "person_id",
    "last_name",
    "first_name",
    "rr_id",
    "status",
    "conditions",
    "eicr_id",
    "eicr_version_number",
    "authoring_datetime",
    "provider_id",
    "facility_id_number",
    "facility_name",
    "facility_type",
    "encounter_type",
    "encounter_start_date",
    "encounter_end_date",
    "active_problem_1",
    "active_problem_date_1",
    "active_problem_2",
    "active_problem_date_2",
    "active_problem_3",
    "active_problem_date_3",
    "active_problem_4",
    "active_problem_date_4",
    "active_problem_5",
    "active_problem_date_5",
    "reason_for_visit",
    "test_type_1",
    "test_result_1",
    "test_result_interp_1",
    "specimen_type_1",
    "performing_lab_1",
    "specimen_collection_date_1",
    "result_date_1",
    "test_type_2",
    "test_result_2",
    "test_result_interp_2",
    "specimen_type_2",
    "performing_lab_2",
    "specimen_collection_date_2",
    "result_date_2"
]

schema_cols = [StructField(f, StringType(), True) for f in schema_fields]

# incident_id can't be null for subsequent joins
schema_cols.append(StructField("incident_id", StringType(), False))
schema = StructType(schema_cols)

df = spark.createDataFrame([], schema)
df.write.format("delta").mode("overwrite").saveAsTable("test_delta_lake")

With the table created and properly formatted with our desired columns, we can create two dummy rows and insert them into the table using spark's appends:

In [ ]:
# Note the tuple around the inserted information--spark additions need to be
# of the form [(,)] to infer struct types
row_1 = [("2c6d5fd1-4a70-11eb-99fd-ad786a821574", "a81bc81b-dead-4e5d-abff-90865d1e13b1", "Shepard", \
         "John", "12-34-56-78", "12", "", "11111111", "2", "", "999", "1", "Huerta Memorial Hospital", \
         "", "encounter", "2021-08-14", "2021-08-16", "", "", "", "", "", "", "", "", "", "", "physical", \
         "", "", "", "", "", "", "", "", "", "", "" ,"", "", "", "123456789")]
row_2 = [("2fdd0b8b-4a70-11eb-99fd-ad786a821574", "a81bc81b-dead-4e5d-abff-90865d1e13b1", "Anderson", \
         "David", "97-56-4862", "24", "", "99999", "168", "2022-12-12", "6d8e9s-98w7szz", "84yfd3556d", \
          "Sunset Strip", "outpatient", "OKI", "2022-12-02", "2022-12-11", "arthritis", "2020-10-10", \
          "", "", "", "", "", "", "", "", "concern", "degenerative disk test", "positive", \
          "patient has a bad back", "vertebrae fluid", "Easy Pete's Discount Disk Checks", "2022-10-10", \
          "", "", "", "", "", "", "", "", "987654321")]
row_1.extend(row_2)

# Accepted practice for row appending is union
row_1 = spark.createDataFrame(row_1, schema)
df = df.union(row_1)
df.show()

df.write.format("delta").mode("overwrite").save("test_delta_lake")